## **Packages**

In [1]:
# packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from hdfs import InsecureClient
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

In [2]:
# matplotlib 한글폰트 
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='D2Coding')
       
# rc('font', family='NanumGothic') # 나눔폰트 사용시

## **DW에 있는 Data 사용하기 위한 Hdfs 클라이언트 객체 생성**

In [3]:
client = InsecureClient('http://localhost:9870', user='lab09')

## **Data 불러오기**
- 관광지 평점 데이터 = tour_ratings.json

In [4]:
ratings = spark.read.json('/raw_data/tour_ratings.json', encoding='utf-8')

In [5]:
ratings.show()

+----------+--------------------------------+------+
|Unnamed: 0|                            명칭|  평점|
+----------+--------------------------------+------+
|         0|                    (주)교보문고|   0.0|
|         1|            1.21 무장공비 침투로|   5.6|
|         2|           2024 코리아그랜드세일|   0.0|
|         3|   24시 마린낚시(전곡항 24 마...| 65.84|
|         4|                        93뮤지엄|240.88|
|         5|                     CICA 미술관|174.16|
|         6|    C아트뮤지엄(숲속의 미술공원)|   7.2|
|         7|                    DMZ 마루캠핑|  58.4|
|         8|                    DMZ 생생누리|   0.0|
|         9|                       HJ글램핑2|   0.0|
|        10|                       JTN아트홀|  49.6|
|        11|  K-일러스트레이션페어 서울 2024|   0.0|
|        12|           K-핸드메이드페어 2024|   0.0|
|        13|             KT&G 상상마당(홍대)|   0.0|
|        14|         KT&G상상마당 대치아트홀|   0.0|
|        15|                      MUSEUM 209|   0.0|
|        16|                     M컨템포러리|   0.0|
|        17|                            TIN

## **1️⃣ 인덱스 생성**
- 관광지 정보 데이터와 index로 fk 를 설정하여 구분하기 위해, 동일한 형태로 맞춰준다.

In [6]:
rating_with_index = ratings.toPandas()

In [7]:
rating_with_index.reset_index(inplace=True)
rating_with_index.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [8]:
rating_with_index = rating_with_index.drop('id', axis=1)
rating_with_index['index'] = rating_with_index['index']+1

In [9]:
rating_with_index.head()
rating_with_index.info()

,index,명칭,평점
0,1,(주)교보문고,0.00
1,2,1.21 무장공비 침투로,5.60
2,3,2024 코리아그랜드세일,0.00
3,4,24시 마린낚시(전곡항 24 마린낚시),65.84
4,5,93뮤지엄,240.88


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   3504 non-null   int64  
 1   명칭      3504 non-null   object 
 2   평점      3504 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 82.2+ KB


In [10]:
# 다시 spark df 로 

# # 'Attribute Error: 'iteritems'가 없다고 나올때
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

spark_rating = spark.createDataFrame(rating_with_index)

In [11]:
type(spark_rating)

pyspark.sql.dataframe.DataFrame

## **✅ 확인**

In [12]:
# 관광지 정보 데이터 불러오기
travel_data = spark.read.json('/dw_data/travel_data_preprocessed.json', encoding='utf-8')
travel_data_pd = travel_data.toPandas()
travel_data_pd.head()
travel_data_pd.info()

,index,개요,개장기간,공연시간,관광지 분류,관람소요시간,명칭,분류,상세정보,쉬는날,...,이용요금,입장료,전화번호,주소,주차시설,주차요금,지역,체험안내,행사시작일,행사종료일
0,1,"교보문고 광화문점은 국민교육 진흥의 실천적 구현, 독서인구 저변 확대를 통한 국민정...",None,None,문화시설,None,(주)교보문고,문화시설,None,설/추석 당일,...,무료,None,None,서울특별시 종로구 종로 1(종로1가),주차 가능,유료 (회원여부/구매금액에 따라 무료주차헤택 적용),서울,None,NaN,NaN
1,2,※ 민통초소에 신분증 제시하여 출입 가능 (25인 이상 출입시 일주일 전 신청서 제...,None,None,인문관광지,None,1.21 무장공비 침투로,역사관광지,화장실:있음\n,매주 화요일,...,None,None,None,경기도 연천군 장남면 반정리 294,None,None,경기도,None,NaN,NaN
2,3,코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화관광축제로...,None,None,축제공연행사,None,2024 코리아그랜드세일,공연행사,행사소개:코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화...,None,...,무료,None,070-7787-4242,서울특별시 종로구 인사동5길 29 태화빌딩802호,None,None,서울,None,2.0240111E7,2.0240229E7
3,4,"경기도 화성시 서신면에 위치해 있는 24시 마린 낚시는 주변에 전곡항, 탄도항, 제...",None,None,레포츠,None,24시 마린낚시(전곡항 24 마린낚시),레포츠,"포인트안내:\n이용요금:선박별 상이 홈페이지 참조\n보유장비: 22인승, 21인승,...",None,...,None,None,None,경기도 화성시 서신면 전곡항로 317,None,None,경기도,None,NaN,NaN
4,5,그림을 열망하던 청년은 첫 월급 3000원을 쪼개 그림 한 점을 구입했다. 당시 가...,None,None,문화시설,None,93뮤지엄,문화시설,None,"매주 월요일(단,월요일이 공휴일인 경우 정상 개관,해당주 화요일 휴관)",...,기간에따라 변동,None,None,경기도 파주시 탄현면 헤이리마을길 59-58헤이리예술마을 G-31,주차가능,None,경기도,None,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          3504 non-null   int64 
 1   개요             3501 non-null   object
 2   개장기간           577 non-null    object
 3   공연시간           30 non-null     object
 4   관광지 분류         3504 non-null   object
 5   관람소요시간         240 non-null    object
 6   명칭             3504 non-null   object
 7   분류             3504 non-null   object
 8   상세정보           3038 non-null   object
 9   쉬는날            2304 non-null   object
 10  애완동물 동반 가능 여부  3504 non-null   object
 11  우편번호           3400 non-null   object
 12  유모차 대여 여부      3504 non-null   object
 13  이용시간           2678 non-null   object
 14  이용시기           11 non-null     object
 15  이용요금           670 non-null    object
 16  입장료            22 non-null     object
 17  전화번호           104 non-null    object
 18  주소             3502 non-null

In [16]:
rating_with_index[(rating_with_index['index'] >= 350) & (rating_with_index['index'] < 400)]

,index,명칭,평점
349,350,노문야영장,0.00
350,351,노산팔경캠핑장,420.64
351,352,노송지대(수원),0.00
352,353,노원 불빛정원,0.00
353,354,노원문화예술회관,9.60
354,355,노원우주학교,0.00
355,356,노을공원,2192.04
356,357,노을빛바다애글램핑,8.00
357,358,노을진 캠핑장,1160.48
358,359,노을캠핑장,532.08


In [15]:
travel_data_pd[['index', '명칭']][(travel_data_pd['index'] >= 350) & (travel_data_pd['index'] < 400)]

,index,명칭
349,350,노문야영장
350,351,노산팔경캠핑장
351,352,노송지대(수원)
352,353,노원 불빛정원
353,354,노원문화예술회관
354,355,노원우주학교
355,356,노을공원
356,357,노을빛바다애글램핑
357,358,노을진 캠핑장
358,359,노을캠핑장


## **👍 파일 저장**

In [17]:
# hdfs에 저장
spark_rating.coalesce(1).write.format('json').save('/dw_data/tour_ratings.json')